In [22]:
from time import sleep

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

url = "https://chicagoengineersfoundation.awardspring.com/"

review_df = {'Reviewer'                 : [],
             'Applicant'                : [],
             'Community Service / Work' : [],
             'Essay'                    : [],
             'Career Goals'             : [],
             'Bonus/Reviewer Discretion': [],
             'Notes'                    : []
             }

review_feedback_df = pd.DataFrame(review_df)

driver = webdriver.Firefox()

In [23]:
review_feedback_df = pd.DataFrame(review_df)


In [24]:
driver.get(f'{url}')

In [25]:
# Login

email_xpath = '//*[@id="UserEmail"]'
email = 'driscoll42@gmail.com'

inputElement = driver.find_element(by=By.XPATH, value=email_xpath)
inputElement.send_keys(email)

pw_xpath = '//*[@id="Password"]'
pw = '$lNwiYF4SYs3s^$!6K$v'

inputElement = driver.find_element(by=By.XPATH, value=pw_xpath)
inputElement.send_keys(pw)

inputElement.send_keys(Keys.ENTER)

In [26]:
# Go to Reviewers
url = "https://chicagoengineersfoundation.awardspring.com/Admin/Users/Reviewers"

driver.get(f'{url}')

In [29]:
def get_reviewer_scores():
    commwork = 0
    essay = 0
    career = 0
    bonus = 0

    for score in range(2, 5):
        scorecard_xpath = f'/html/body/section/div/main/div/div/div/div[6]/div[2]/div[{score}]/div/table/tbody/tr/td[1]'
        element = driver.find_element(by=By.XPATH, value=scorecard_xpath)
        inner_html = element.get_attribute('innerHTML')
        start = inner_html.find('value="') + len('value="')

        end = inner_html.find('">')

        substring = inner_html[start:end]
        if score == 2:
            commwork = int(substring)
        elif score == 3:
            essay = int(substring)
        elif score == 4:
            bonus = int(substring)

    note_xpath = '//*[@id="notes"]'
    note_ele = driver.find_element(by=By.XPATH, value=note_xpath)
    notes = note_ele.text
    return commwork, essay, career, bonus, notes


In [32]:
impersonate_user_xpath = '/html/body/section/div/main/div/div[1]/span[2]/input'
impersonate_user_xpath = '//*[@id="impersonate"]'


def get_reviewer_feedback(review_feedback_df, num_reviewers, start_pos):
    sleep(2)
    for x in range(start_pos, num_reviewers):  # TODO: calculate number
        url = "https://chicagoengineersfoundation.awardspring.com/Admin/Users/Reviewers"

        driver.get(f'{url}')
        sleep(1)

        next_page_xpath = '/html/body/section/div/main/div/div[2]/ul/li[6]/a/span'
        inputElement = driver.find_element(by=By.XPATH, value=next_page_xpath)
        inputElement.click()
        sleep(1)

        re_xpath = f'/html/body/section/div/main/div/div[2]/table/tbody/tr[{x}]/td[1]/a'

        print(re_xpath)
        inputElement = driver.find_element(by=By.XPATH, value=re_xpath)
        reviewer_name = inputElement.text
        inputElement.send_keys(Keys.ENTER)
        sleep(1)
        # Impersonate Reviewer
        try:
            inputElement = driver.find_element(by=By.XPATH, value=impersonate_user_xpath)
            inputElement.click()
            sleep(2)
        except Exception as e:
            sleep(4)
            inputElement = driver.find_element(by=By.XPATH, value=impersonate_user_xpath)
            inputElement.click()
            sleep(2)

        student_xpath = f'/html/body/section/div/main/div/div/div[2]/table/tbody/tr[1]/td[2]/a'
        try:
            inputElement = driver.find_element(by=By.XPATH, value=student_xpath)
            inputElement.send_keys(Keys.ENTER)
            sleep(2)
            more_students = True

            while more_students:
                student_name_xpath = '/html/body/section/div/main/div/div/div/div[3]/h2'
                student_name = driver.find_element(by=By.XPATH, value=student_name_xpath).text

                commwork, essay, career, bonus, notes = get_reviewer_scores()
                # print(student_name, commwork, essay, career, bonus, notes)
                new_review_df = {'Reviewer'                  : reviewer_name,
                                 'Applicant'                 : student_name,
                                 'Community Service / Work'  : commwork,
                                 'Short Essay'               : essay,
                                 'Bonus/Discretionary Points': bonus,
                                 'Notes'                     : notes
                                 }

                review_feedback_df = pd.concat([review_feedback_df, pd.DataFrame.from_records([new_review_df])])

                try:
                    next_button_xpath = '/html/body/section/div/main/div/div/div/div[2]/a[1]'
                    next_button_element = driver.find_element(by=By.XPATH, value=next_button_xpath)
                    next_button_element.send_keys(Keys.ENTER)
                    sleep(2)
                except Exception as e:
                    more_students = False
            sleep(2)
        except Exception as e:
            pass
        sleep(1)
        # Stop Impersonating
        print(reviewer_name)
        try:
            stop_impers_xpath = '//*[@id="stopImpersonate"]'
            stop_impers_element = driver.find_element(by=By.XPATH, value=stop_impers_xpath)
            stop_impers_element.click()
            sleep(2)
        except Exception as e:
            sleep(5)
            stop_impers_xpath = '//*[@id="stopImpersonate"]'
            stop_impers_element = driver.find_element(by=By.XPATH, value=stop_impers_xpath)
            stop_impers_element.click()
            sleep(2)
    return review_feedback_df


In [31]:
review_feedback_df = get_reviewer_feedback(review_feedback_df, 26, 1)


/html/body/section/div/main/div/div[2]/table/tbody/tr[1]/td[1]/a
Abduljelil, Yasmin 3 40 0 0 
Agrawal, Nikita 20 55 0 20 very high achiever, lots of community service. great personal website and extra curriculars. 
Alonge-Oludaye, Tunmise 5 50 0 15 
Ambrose , Samantha 12 35 0 5 their essay and their resume seem like they were written by two completely different people. The sentence structure, vocabulary and over all grammar are way off between the two
Arambula, Aracely 12 55 0 12 graduating high school with an associates degree. 
Becker, Luke 18 55 0 16 eagle scout candidate
Bermeo, Nikolay 6 40 0 5 transcript not provided
Cerritos, Jafet 4 44 0 4 the essay is very different than the writing structure of the other questions. 
Cosentino, Marcus 0 0 0 0 GPA does not meet the minimum 2.75, did not complete the review
Del Llano, Danny 10 50 0 10 
Devarajan, Sanjana 19 54 0 18 strong resume, strong list of awards/accomplishments plus played a varsity sport
missing transcript, missing accept

In [10]:
review_feedback_df.drop_duplicates(inplace=True)
display(review_feedback_df.head(10))
review_feedback_df.to_excel('2024 CEF Reviewer Detailed Feedback.xlsx')
review_feedback_df = pd.read_excel('2024 CEF Reviewer Detailed Feedback.xlsx')

,Reviewer,Applicant,Community Service / Work,Essay,Career Goals,Bonus/Reviewer Discretion,Notes,Short Essay,Bonus/Discretionary Points
0,"Avram, Joe","Abduljelil, Yasmin",3.0,NaN,NaN,NaN,,40.0,0.0
0,"Avram, Joe","Agrawal, Nikita",20.0,NaN,NaN,NaN,"very high achiever, lots of community service....",55.0,0.0
0,"Avram, Joe","Alonge-Oludaye, Tunmise",5.0,NaN,NaN,NaN,,50.0,0.0
0,"Avram, Joe","Ambrose , Samantha",12.0,NaN,NaN,NaN,their essay and their resume seem like they we...,35.0,0.0
0,"Avram, Joe","Arambula, Aracely",12.0,NaN,NaN,NaN,graduating high school with an associates degr...,55.0,0.0
0,"Avram, Joe","Becker, Luke",18.0,NaN,NaN,NaN,eagle scout candidate,55.0,0.0
0,"Avram, Joe","Bermeo, Nikolay",6.0,NaN,NaN,NaN,transcript not provided,40.0,0.0
0,"Avram, Joe","Cerritos, Jafet",4.0,NaN,NaN,NaN,the essay is very different than the writing s...,44.0,0.0
0,"Avram, Joe","Cosentino, Marcus",0.0,NaN,NaN,NaN,"GPA does not meet the minimum 2.75, did not co...",0.0,0.0
0,"Avram, Joe","Del Llano, Danny",10.0,NaN,NaN,NaN,,50.0,0.0


In [9]:
review_feedback_df = get_reviewer_feedback(review_feedback_df, 26, 1)

/html/body/section/div/main/div/div[2]/table/tbody/tr[1]/td[1]/a
Avram, Joe
/html/body/section/div/main/div/div[2]/table/tbody/tr[2]/td[1]/a
B, Debbie
/html/body/section/div/main/div/div[2]/table/tbody/tr[3]/td[1]/a
Bennett, Sid
/html/body/section/div/main/div/div[2]/table/tbody/tr[4]/td[1]/a
Beuving, Henry
/html/body/section/div/main/div/div[2]/table/tbody/tr[5]/td[1]/a
Birrell, Debbie
/html/body/section/div/main/div/div[2]/table/tbody/tr[6]/td[1]/a
Burns, Pat
/html/body/section/div/main/div/div[2]/table/tbody/tr[7]/td[1]/a
Candiano, Charles
/html/body/section/div/main/div/div[2]/table/tbody/tr[8]/td[1]/a
Cempel, Erik
/html/body/section/div/main/div/div[2]/table/tbody/tr[9]/td[1]/a
Clemente, Marvi
/html/body/section/div/main/div/div[2]/table/tbody/tr[10]/td[1]/a
Cueto, Victor
/html/body/section/div/main/div/div[2]/table/tbody/tr[11]/td[1]/a
Deitch, Sabrina
/html/body/section/div/main/div/div[2]/table/tbody/tr[12]/td[1]/a
Dixon, La Toya
/html/body/section/div/main/div/div[2]/table/tbod

In [33]:
review_feedback_df = get_reviewer_feedback(review_feedback_df, 11, 1)


/html/body/section/div/main/div/div[2]/table/tbody/tr[1]/td[1]/a
Reviewer 1, College
/html/body/section/div/main/div/div[2]/table/tbody/tr[2]/td[1]/a
Reviewer 2, College
/html/body/section/div/main/div/div[2]/table/tbody/tr[3]/td[1]/a
Reviewer 3, College
/html/body/section/div/main/div/div[2]/table/tbody/tr[4]/td[1]/a
Rienks, Steve
/html/body/section/div/main/div/div[2]/table/tbody/tr[5]/td[1]/a
Riley, Tom
/html/body/section/div/main/div/div[2]/table/tbody/tr[6]/td[1]/a
Agrawal, Nikita 20 58 0 20 Very impressive record of work and volunteering.
Articulate.  Motivated. An excellent candidate.
Del Llano, Danny 5 55 0 5 Enthusiastic about his area of interest.    Weak on volunteer work, although he sees his interest in flying cars being something that will improve urban life.
Gray , Gabrielle 0 55 0 0 Sincere and introspective.   Weak grades.    Limited stem courses.   Undefined goals.
Smith, Mitchell 20 60 0 10 I urge you to accept this candidate!    He is creative, self-motivated, cente

In [17]:
review_feedback_df = pd.read_excel('2024 CEF Reviewer Detailed Feedback.xlsx')
review_feedback_second_df = get_reviewer_feedback(review_feedback_df, 11, 1)

/html/body/section/div/main/div/div[2]/table/tbody/tr[1]/td[1]/a
Reviewer 1, College
/html/body/section/div/main/div/div[2]/table/tbody/tr[2]/td[1]/a
Reviewer 2, College
/html/body/section/div/main/div/div[2]/table/tbody/tr[3]/td[1]/a
Reviewer 3, College
/html/body/section/div/main/div/div[2]/table/tbody/tr[4]/td[1]/a
Rienks, Steve
/html/body/section/div/main/div/div[2]/table/tbody/tr[5]/td[1]/a
Riley, Tom
/html/body/section/div/main/div/div[2]/table/tbody/tr[6]/td[1]/a
Rubert, Ellen
/html/body/section/div/main/div/div[2]/table/tbody/tr[7]/td[1]/a
Thomas, George
/html/body/section/div/main/div/div[2]/table/tbody/tr[8]/td[1]/a
Treleaven, Jim
/html/body/section/div/main/div/div[2]/table/tbody/tr[9]/td[1]/a
Tuladhar, Sasu
/html/body/section/div/main/div/div[2]/table/tbody/tr[10]/td[1]/a
Weaver, Linda


In [35]:
# Merge review_feedback_df and review_feedback_second_df
review_feedback_df = pd.concat([review_feedback_df, review_feedback_second_df])
review_feedback_df.drop_duplicates(inplace=True)
display(review_feedback_df.head(10))
review_feedback_df.to_excel('2024 CEF Reviewer Detailed Feedback.xlsx')

,Reviewer,Applicant,Community Service / Work,Essay,Career Goals,Bonus/Reviewer Discretion,Notes,Short Essay,Bonus/Discretionary Points,Unnamed: 0
0,"Avram, Joe","Abduljelil, Yasmin",3.0,NaN,NaN,NaN,,40.0,0.0,NaN
0,"Avram, Joe","Agrawal, Nikita",20.0,NaN,NaN,NaN,"very high achiever, lots of community service....",55.0,20.0,NaN
0,"Avram, Joe","Alonge-Oludaye, Tunmise",5.0,NaN,NaN,NaN,,50.0,15.0,NaN
0,"Avram, Joe","Ambrose , Samantha",12.0,NaN,NaN,NaN,their essay and their resume seem like they we...,35.0,5.0,NaN
0,"Avram, Joe","Arambula, Aracely",12.0,NaN,NaN,NaN,graduating high school with an associates degr...,55.0,12.0,NaN
0,"Avram, Joe","Becker, Luke",18.0,NaN,NaN,NaN,eagle scout candidate,55.0,16.0,NaN
0,"Avram, Joe","Bermeo, Nikolay",6.0,NaN,NaN,NaN,transcript not provided,40.0,5.0,NaN
0,"Avram, Joe","Cerritos, Jafet",4.0,NaN,NaN,NaN,the essay is very different than the writing s...,44.0,4.0,NaN
0,"Avram, Joe","Cosentino, Marcus",0.0,NaN,NaN,NaN,"GPA does not meet the minimum 2.75, did not co...",0.0,0.0,NaN
0,"Avram, Joe","Del Llano, Danny",10.0,NaN,NaN,NaN,,50.0,10.0,NaN


In [39]:
# Note: NEed to start on the Reviewers Screen
review_feedback_df = get_reviewer_feedback(review_feedback_df, 26, 1)
for next_num in range(1, 14):
    next_rebutton_xpath = '/html/body/section/div/main/div/div[2]/div/ul/li[6]/a/span'
    next_rebutton_element = driver.find_element(by=By.XPATH, value=next_rebutton_xpath)
    next_rebutton_element.click()
    sleep(2)
    review_feedback_df = get_reviewer_feedback(review_feedback_df, next_num + 1, start_pos=next_num)

/html/body/section/div/main/div/div[2]/table/tbody/tr[1]/td[1]/a
Avram, Joe
/html/body/section/div/main/div/div[2]/table/tbody/tr[2]/td[1]/a
B, Debbie
/html/body/section/div/main/div/div[2]/table/tbody/tr[3]/td[1]/a
Bennett, Sid
/html/body/section/div/main/div/div[2]/table/tbody/tr[4]/td[1]/a
Beuving, Henry
/html/body/section/div/main/div/div[2]/table/tbody/tr[5]/td[1]/a
Birrell, Debbie
/html/body/section/div/main/div/div[2]/table/tbody/tr[6]/td[1]/a
Burns, Pat
/html/body/section/div/main/div/div[2]/table/tbody/tr[7]/td[1]/a
Candiano, Charles
/html/body/section/div/main/div/div[2]/table/tbody/tr[8]/td[1]/a
Cempel, Erik
/html/body/section/div/main/div/div[2]/table/tbody/tr[9]/td[1]/a
Clemente, Marvi
/html/body/section/div/main/div/div[2]/table/tbody/tr[10]/td[1]/a
Cueto, Victor
/html/body/section/div/main/div/div[2]/table/tbody/tr[11]/td[1]/a
Deitch, Sabrina
/html/body/section/div/main/div/div[2]/table/tbody/tr[12]/td[1]/a
Dixon, La Toya
/html/body/section/div/main/div/div[2]/table/tbod

NoSuchElementException: Message: Unable to locate element: /html/body/section/div/main/div/div[2]/div/ul/li[6]/a/span
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [13]:
review_feedback_df.drop_duplicates(inplace=True)
display(review_feedback_df.head(10))
review_feedback_df.to_excel('2023 CEF Reviewer Detailed Feedback.xlsx')

,Reviewer,Applicant,Community Service / Work,Essay,Career Goals,Bonus/Reviewer Discretion,Notes
0,"Avram, Joe","Abraham, Devin",9.0,36.0,28.0,17.0,"started a student engineering organization, ha..."
0,"Avram, Joe","Abuelhija, Haron",7.0,38.0,28.0,15.0,
0,"Avram, Joe","Barrera, Jesus",9.0,34.0,27.0,17.0,
0,"Avram, Joe","Barron, Ronan",7.0,37.0,27.0,20.0,
0,"Avram, Joe","Biesiada, Francisco",9.0,20.0,15.0,6.0,"did not upload transcript, reuploaded acceptan..."
0,"Avram, Joe","Cabello, Kayla",8.0,40.0,28.0,18.0,very well written and has a clear goal and und...
0,"Avram, Joe","Ceja Jr. , Gilberto",5.0,24.0,20.0,10.0,"community service hours say 20,000 (that is 2...."
0,"Avram, Joe","De La Rosa, Miguel",8.0,30.0,25.0,14.0,spelling errors in the essay. it was short but...
0,"Avram, Joe","Dedic, Daniel",0.0,32.0,22.0,12.0,
0,"Avram, Joe","Di Franco, Giuliano",9.0,36.0,27.0,17.0,
